# From Mantid to Scipp
## Data types
### Workspaces

| Mantid | Scipp |
| ---| --- |
| `Workspace2D` | `DataArray` |
| `EventWorkspace` | `DataArray` |
| `WorkspaceSingleValue` | `DataArray` |
| `MDHistoWorkspace` | `DataArray` |
| `MDEventWorkspace` | not supported |
| `TableWorkspace` | `Dataset` |
| `WorkspaceGroup` | `Dataset` (aligned dimensions), otherwise Python `list` or `dict` |

#### Notes
- In many cases it may be desirable to use `Dataset` instead of `DataArray`. You can easily create a `Dataset` directly from a `DataArray`.
- Scipp takes basic geometric information from Mantid's instrument in the form of positions. Detector grouping by spectrum is respected. Upon conversion, Scipp will perform spherical coordinate averaging for the group based on the beam direction, this preserves the average scattering angle between a group of detectors and the spectra respresenting the group. This may yield slightly different detector and spectrum positions between what is natively stored in Mantid's instrument and Scipp.
- Run and Sample are copied over to scipp from any MatrixWorkspace derived Workspaces.
- Scipp (or rather conversion to scipp) is currently still incomplete and does not carry over all information from a workspace.

### Other

| Mantid | Scipp |
| ---| --- |
| `DetectorInfo` | `Dataset` |

## Concepts

Mantid's `MatrixWorkspace` (the common base class of `Workspace2D` and `EventWorkspace`) uses the terms "X", "Y", and "E" to refer to one of its axes, the data values, and the uncertainties.

- Mantid stores **standard-deviations** in "E", whereas scipp stores **variances**.
- Typically Mantid's "X" is the coordinate axis for the time-of-flight dimension, or the dimension derived from it.
- Mantid's "Y" is not the axis for the second dimension, but the **data**.
- Mantid's "X", "Y", and "E" are 1-D arrays of 1-D arrays, whereas scipp stores 2-D (or higher) arrays, if applicable.

We have the following "equivalence":

| Mantid | Scipp | comment |
| ---| --- | --- |
| `ws.readY(i)` | `data.values` |
| `ws.readE(i)` | `data.variances` | square former, or `sqrt` latter |
| `ws.readX(i)` | `data.coords['tof'].values` | dimension label may vary |
| `ws.getAxis(0).getUnit()` | `data.coords['tof'].unit` | dimension label may vary |
| `ws.getAxis(1)` | `data.coords['spectrum']` | dimension label may vary |

Here `i` is the index along the second axis (axis index `1`).
Mantid's `readX`, `readY`, and `readE` always return 1-D arrays, whereas the `values` and `variances` properties in scipp return a multi-dimensional array.
That is, there is no actual equivalence.


## Algorithms

### Notes
- In **Mantid** a Python variable referencing **a workspace is** under the hood **a global variable**.
  Unless specified otherwise the variable name is the name of the workspace in the [AnalysisDataService](https://docs.mantidproject.org/nightly/concepts/AnalysisDataService.html).
  For marginally more clarity, the examples in the following therefore use the string-based syntax for specifying output workspaces.
  *In scipp there is no such limitation and everything behaves just like normal variables in Python.*
- Unless stated otherwise, the following code examples assume datasets or data arrays have `'tof'` for what Mantid calls "X" and `'spectrum'` why Mantid calls "Y" or "spectrum axis".
- There is no strict 1:1 equivalence between Mantid workspaces and functionality in scipp.
  The examples below give the most common examples, but in many cases exceptions apply and detailed behavior may differ.
  If in doubt, consult the Mantid algorithm documentation and the scipp documentation.

In [ ]:
import mantid.simpleapi as mantid
import scipp as sc
import numpy as np

### Test Data

In [ ]:
input_bin_edges = mantid.CreateSampleWorkspace()
input_point_data = mantid.ConvertToPointData(input_bin_edges)
a_mantid = mantid.ExtractSpectra(input_point_data, StartWorkspaceIndex=0, EndWorkspaceIndex=10)
b_mantid = mantid.ExtractSpectra(input_point_data, StartWorkspaceIndex=10, EndWorkspaceIndex=20)
a_scipp = sc.neutron.from_mantid(a_mantid)
b_scipp = sc.neutron.from_mantid(b_mantid)

### Generic algorithms

#### CloneWorkspace

In [ ]:
cloned = mantid.CloneWorkspace(InputWorkspace=input_point_data, OutputWorkspace='copy')
cloned

Equivalent in scipp:

In [ ]:
copy = a_scipp.copy()
copy

#### DeleteWorkspace

In [ ]:
mantid.DeleteWorkspace(Workspace=cloned)

Equivalent in scipp:

In [ ]:
del copy

#### ExtractSingleSpectrum

In [ ]:
spec = mantid.ExtractSingleSpectrum(
    InputWorkspace=input_point_data,
    OutputWorkspace='spec',
    WorkspaceIndex=7)

Equivalent in scipp:

In [ ]:
spec = a_scipp['spectrum', 7]

If an actual *copy* is required use:

In [ ]:
spec = a_scipp['spectrum', 7].copy()

#### ExtractSpectra / CropWorkspace

In [ ]:
mantid.ExtractSpectra(
    InputWorkspace=input_point_data,
    OutputWorkspace='spectra',
    StartWorkspaceIndex=1,
    EndWorkspaceIndex=4)

Equivalent in scipp:

In [ ]:
a_scipp['spectrum', 1:5]

If an actual *copy* is required use:

In [ ]:
spectra = a_scipp['spectrum', 1:5].copy()

#### Transpose

In [ ]:
mantid.Transpose(InputWorkspace=input_point_data, OutputWorkspace='data')

Equivalent in scipp:
Transposing is *implicit* and automatic based on dimension labels and not required for any of the common operations, including plotting.

#### AppendSpectra

In [ ]:
mantid.AppendSpectra(
    InputWorkspace1=a_mantid,
    InputWorkspace2=b_mantid, OutputWorkspace='combined')

Equivalent in scipp:

In [ ]:
data = sc.concatenate(a_scipp, b_scipp, 'spectrum')

#### ConjoinXRuns

In [ ]:
mantid.ConjoinXRuns(
    InputWorkspaces=['a_mantid','b_mantid'],
    OutputWorkspace='data')

Equivalent in scipp:

In [ ]:
sc.concatenate(a_scipp, b_scipp, 'tof')

#### ConjoinSpectra

In [ ]:
mantid.ConjoinSpectra(
    InputWorkspaces='a_mantid, b_mantid',
    OutputWorkspace='out',
    WorkspaceIndex=7)

Equivalent in scipp:

In [ ]:
sc.concatenate(a_scipp, b_scipp, 'spectra')['spectrum', 7]

Or more efficiently

In [ ]:
sc.concatenate(a_scipp['spectrum', 7], b_scipp['spectrum', 7], 'spectrum')

#### GroupWorkspaces

In [ ]:
mantid.GroupWorkspaces(
    InputWorkspaces='a_mantid, b_mantid',
    OutputWorkspace='data')

Equivalent in scipp:

In [ ]:
sc.Dataset({
    'data1':a_scipp,
    'data2':a_scipp.copy()})

This requires aligned dimensions (matching coordinates) in all input arrays. 
It is a more powerful concept than that provided by WorkspaceGroups, but restricted. 
Slicing for example can be applied to the whole dataset and items are handled accordingly. 
For a loose collection of objects, more similar to the WorkspaceGroup concept, 
use a Python `dict` or `list` for grouping unaligned data.

#### Rebin  `Workspace2D` into `Workspace2D`

In [ ]:
mantid.Rebin(
    InputWorkspace=input_point_data,
    OutputWorkspace='histo',
    Params='0,100,20000')

Equivalent in scipp:

In [ ]:
edges = sc.Variable(
    ['tof'],
    values=np.arange(0.0, 20000.0, 100.0),
    unit=sc.units.us)
data = sc.neutron.from_mantid(input_bin_edges)
sc.rebin(data, 'tof', edges)

#### Rebin  `EventWorkspace` preserving events

In [ ]:
event_workspace = mantid.CreateSampleWorkspace(WorkspaceType='Event')

mantid.Rebin(
    InputWorkspace=event_workspace,
    OutputWorkspace='rebinned_events',
    Params='0,100,20000',
    PreserveEvents=True)

Equivalent in scipp:

In [ ]:
events = sc.neutron.from_mantid(event_workspace)
tof_edges = sc.Variable(
    ['tof'],
    values=np.arange(0.0, 20000.0, 100.0),
    unit=sc.units.us)

sc.bin(events, [tof_edges])

#### Rebin  `EventWorkspace` into `Workspace2D`

In [ ]:
mantid.Rebin(
    InputWorkspace=event_workspace,
    OutputWorkspace='histo',
    Params=[0,100,20000],
    PreserveEvents=False)

Equivalent in scipp:

In [ ]:
tof_edges = sc.Variable(
    ['tof'],
    values=np.arange(0.0, 20000.0, 100.0),
    unit=sc.units.us)
sc.histogram(events, edges)

#### Rebin with logarithmic bins

In [ ]:
mantid.Rebin(
    InputWorkspace=event_workspace,
    OutputWorkspace='histo',
    Params='2,-0.035,10')

Equivalent in scipp:

In [ ]:

edges = sc.Variable(
    ['tof'],
    values=np.geomspace(start=2, stop=10, num=100),
    unit=sc.units.us)
sc.bin(events, [edges])


Bin edges in scipp can be created from an arbitrary array with increasing values, the use of `numpy.geomspace` is simply one example for generating bins spaced evenly on a log scale.

<div class="alert alert-info">

**Note**

Both scipp and Mantid support binary and in-place operations such as + and +=.
Mantid's binary operations call underlying algorithms as part of their implementation.
This makes it difficult to change some default behaviour, for example if you want to prevent output workspaces from being registered in Mantid's Analysis Data Service.

</div>

#### Scale (multiplication)

In [ ]:
mantid.Scale(
    InputWorkspace=input_point_data,
    OutputWorkspace=input_point_data, 
    Factor=7.5,
    Operation="Multiply")

Equivalent in scipp:

In [ ]:
a_scipp *= 7.5
a_scipp

#### Scale (addition)

In [ ]:
mantid.Scale(
    InputWorkspace=input_point_data,
    OutputWorkspace='summed', 
    Factor=7.5,
    Operation="Add")

Equivalent in scipp:

In [ ]:
a_scipp += 7.5 * sc.units.counts
a_scipp

If the data is not dimensionless, the correct unit must be specified:

In [ ]:
a_scipp.data.unit = sc.units.us
try:
    a_scipp += 7.5
except RuntimeError as err:
    print(str(err))
a_scipp += 7.5 * sc.units.us # This is fine now RHS has units

Mantid does not have this safety net.

#### ScaleX

In [ ]:
mantid.ScaleX(
    InputWorkspace=input_point_data,
    OutputWorkspace='output', 
    Factor=7.5,
    Operation="Multiply")

Equivalent in scipp:

In [ ]:
data.coords['tof'] *= 7.5

#### SumSpectra

In [ ]:
mantid.SumSpectra(
    InputWorkspace=input_point_data,
    OutputWorkspace='summed',
    StartWorkspaceIndex=7,
    EndWorkspaceIndex=88)

Equivalent in scipp:

In [ ]:
sc.sum(a_scipp['spectrum', 7:89], 'spectrum')

### Neutron-scattering specific algorithms
#### ConvertUnits

In [ ]:
mantid.ConvertUnits(InputWorkspace=input_point_data, OutputWorkspace='dspacing', Target='dSpacing')

Equivalent in scipp:

In [ ]:
sc.neutron.convert(a_scipp, 'tof', 'd-spacing')

<div class="alert alert-info">

**Note**
    
scipp has no equivalent to the `EMode` and `EFixed` settings of `ConvertUnits`.
Instead, this information is read from the input data, if available (note that currently only elastic scattering is supported).
    
</div>

<div class="alert alert-info">

**Note**

The `scipp.neutron module` also provides a `to_mantid` function, which has limited support for converting scipp data to Mantid workspaces.
Because scipp offers a more flexible container than a Workspace, in particular MatrixWorkspace, it is not always possible to exactly convert all information to Mantid workspaces.

</div>